# Tree species image classification using Artificial Neural Networks

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import os
import matplotlib.pyplot as plt

## Getting the data

In [8]:
# DATA SET DIRECTORIES
local_path = "/home/sciapps/Documents/Repos/tfm"
source_dir = "data/images/image_preprocessing/processed_images/"
train_dir = os.path.join(local_path, source_dir, "train")
test_dir = os.path.join(local_path, source_dir, "test")
val_dir = os.path.join(local_path, source_dir, "val")

In [9]:
# LABELS
class_names = sorted(os.listdir(train_dir))
print(class_names)

['Acer_campestre', 'Alnus_glutinosa', 'Betula_pendula', 'Betula_pubescens', 'Buxus_sempervirens', 'Carpinus_betulus', 'Cornus_sanguinea', 'Corylus_avellana', 'Crataegus_laevigata', 'Crataegus_monogyna', 'Euonymus_europaea', 'Fagus_sylvatica', 'Frangula_alnus', 'Fraxinus_excelsior', 'Ilex_aquifolium', 'Juniperus_communis', 'Malus_sylvestris', 'Pinus_sylvestris', 'Populus_nigra', 'Populus_tremula', 'Prunus_avium', 'Prunus_padus', 'Prunus_spinosa', 'Quercus_petraea', 'Quercus_robur', 'Rhamnus_cathartica', 'Salix_caprea', 'Salix_cinerea_subsp._oleifolia', 'Salix_fragilis', 'Salix_pentandra', 'Salix_viminalis', 'Sambucus_nigra', 'Sorbus_aucuparia', 'Sorbus_rupicola', 'Sorbus_torminalis', 'Taxus_baccata', 'Tilia_cordata', 'Tilia_platyphyllos', 'Tilia_x_europaea', 'Ulmus_glabra', 'Ulmus_procera', 'Viburnum_opulus']


## Image decodification and augmentation

Aim: increase the number of examples by randomly applying transformations to the original images. It also prevents overfitting of the model. 

Augmnetation methods:

- Rotation.
- Width shift.
- Height shift.
- Horizontal flip
- Zooming

`ImageDataGenrator`:

- Read images from the disk.
- Decode images in arrays of float pixel values (here RGB).
- Rescale the floats in the arrays from values between 0 and 255 to 0 and 1.
- Perform real-time image augmentation.

`flow_from_directory`:

- Generate the batches of array image data (aka tensors) with the real-time data augmentation defined in the `ImageDataGenerator`.
- Resize the arrays.

In [4]:
# CONFIGURATION
# Parameters for ImageDataGenerator 
color_mode= "rgb"   
img_height = 28 
img_width = 28   
class_mode="categorical" 
batch_size = 3 
epochs = 15                                     
shuffle=True                                                               
seed = 1234 
# Parameters for test dataset 
test_batch_size = 1

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                            rotation_range=45,
                                            width_shift_range=.15,
                                            height_shift_range=.15,
                                            horizontal_flip=True,
                                            zoom_range=0.5,
                                            validation_split = 0.16)  

In [6]:
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                            target_size=(img_width, img_height),
                                            color_mode = color_mode,
                                            batch_size = batch_size,                               
                                            shuffle = shuffle,
                                            class_mode = class_mode,
                                            subset = "training",
                                            seed=seed
                                            ) 
validation_generator = train_datagen.flow_from_directory(train_dir,  # same directory as training data
                                                    target_size=(img_width, img_height),
                                                    color_mode = color_mode,
                                                    batch_size=batch_size,
                                                    class_mode= class_mode,
                                                    subset='validation',
                                                    seed=seed) # set as validation data 

Found 374 images belonging to 42 classes.
Found 48 images belonging to 42 classes.


## Creating, training and evaluating the model

### Model network architecture

The simplest network architecture constists of 3 layers:
- Input layer, with a number of nodes equal to the number of features in the model.
- Hidden layer, with a variable number of nodes. 
- Output layer, with a number of nodes equal to the number of classes. 

#### The hidden layers

The hidden layers can have different characteristics depending of their use. 
For image classification, at least one of the hidden layers will be convolutional.

##### The convolutional layer

The main characteristic of a convolutional layer is that it applies a filter to each of the elements of a matrix (the pixels of an image). This filter is called the **kernel**. The kernel is a matrix (generally of small size, 2x3, 3x2, 3x3...) with a set of fixed real numbers. Each pixel of the original image is multiplied by the kernel matrix and the result sumed up to output another pixel value for the transformed image. Each time the filter is applied to all the pixels of an image is called a **convolution**.

At this level , the performance of the image feature extraction depends on the values in the kernel and the concatenation of convolutional layers. This is because different filters may be specialized in extracting different features (for example, vertical or horizontal edges) and the sequential input and output values for each layer improves the final output.

In keras, the convolutional layer applied to a 2D matrix is called `Conv2D`.

##### The fully connected layer

A fully connected layer is an all purpose layer where each node receive the inputs from all the nodes from the previous layer, multiplied by their weights, sumed and transformed by the activation funcion.

In keras, the fully connected layer is called `Dense`.



### Creating the model with `Keras`

In `Keras` the model is defined with the `Sequential` method as a linear stack ot layers. The **input layer** is implicit in the first layer (a network with 3 layers will have 2 in `keras Sequential` method).

Ty
the number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64).
    ## Typically, as the width and height shrink, you can afford (computationally) to add more output channels in each Conv2D layer
    # For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.
    # The tf.nn.softmax function converts these logits to "probabilities" for each class

In [25]:
print(train_generator[0].shape())

AttributeError: 'tuple' object has no attribute 'shape'

In [34]:
# MODEL PARAMETERS
#input_shape = 
## Dense layer
array_size = img_width*img_height
dense_size = 10 
num_classes = len(class_names)
## Conv2D layer
kernel_size = (3,3) # number of convolutional filters (width*height of the filter mask)
padding = "same" # case insensitive. Other option: "valid"

In [42]:
# MODEL ARCHITECTURE
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(3,3, padding = padding, activation = "relu", input_shape=(28,28,3)))
model.add(tf.keras.layers.Dense(num_classes, activation = "softmax")) # last layer
#model.build()

In [43]:
# COMPILING THE MODEL
# SparseCategoricalCrossentropi directly uses classes labels,
## so that they don't need to be numerically encoded.
optimizer = "adam" # Options: "sgd", "adam"
model.compile(optimizer=optimizer,
            loss = "sparse_categorical_crossentropy",
            #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 3)         84        
_________________________________________________________________
dense_15 (Dense)             (None, 28, 28, 42)        168       
Total params: 252
Trainable params: 252
Non-trainable params: 0
_________________________________________________________________


The parameters represent the **weights**. 
A weight is a number that multiplies the value of the **input node** before passing it to the **output node** in the next layer. 
An output node receive the values from each of the input nodes multiplied by their weights, after they have been sumed and transformed by the **activation function** into a value between 0 and 1. 

In [44]:
# TRAINING THE MODEL
model.fit_generator(
    train_generator,
    steps_per_epoch=3, # batch_size,
    epochs=15,
    verbose=1, # get a progress bar and ETA
    validation_data=validation_generator,
    validation_steps=2 # batch_size
)

Epoch 1/15
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


InvalidArgumentError: Can not squeeze dim[1], expected a dimension of 1, got 42
	 [[{{node metrics_8/acc/Squeeze}}]]